In [6]:
import pandas as pd
word_dic = pd.read_csv('word_dictionary_ver6.csv', dtype = str)

# 양쪽에 같은 단어가 있다면, 모호성이 크다고 보고 df에서 삭제해줄 것
# ex) 양족 모두 있을 법한 '금리' 라는 단어는 삭제해도 됨

# 1. 둘을 다른 데이터 프레임으로 분리한다. dovish / hawkish
df_hawkish = pd.DataFrame(word_dic['hawkish'], dtype=str)
df_dovish = pd.DataFrame(word_dic['dovish'], dtype=str)

# 2. 둘을 비교해가면서 중복으로 들어간 데이터는 두 df 모두에서 삭제 해 본다.

# 2-1. 우선 각 df 내에서 중복된 값을 지움
df_hawkish = df_hawkish.drop_duplicates()
df_dovish = df_dovish.drop_duplicates()
# 2-2. 둘을 순회하면서 서로 중복된 값을 지운다.
common_values = set(df_hawkish['hawkish']).intersection(set(df_dovish['dovish']))

# 교집합에 해당하는 값 삭제
df_hawkish = df_hawkish[~df_hawkish['hawkish'].isin(common_values)]
df_dovish = df_dovish[~df_dovish['dovish'].isin(common_values)]

# 인덱스를 0부터 재설정
df_hawkish = df_hawkish.reset_index(drop=True)
df_dovish = df_dovish.reset_index(drop=True)

# 3. 개수를 확인하고, 순서를 확인해서 구축한다.
df_hawkish.dropna(inplace=True)
df_dovish.dropna(inplace=True)

In [7]:
df_dovish.head()

,dovish
0,잔류
1,탄핵_정국
2,기준금리_인하_결정
3,조기_금리_인상_우려
4,한국은행_금융통화위원회_기준금리_인하


In [8]:
df_hawkish.head()

,hawkish
0,중국_봉쇄
1,빅_스텝_단행
2,p500지수_기록
3,아세안
4,중심_s


In [9]:
result_df = pd.concat([df_hawkish, df_dovish], axis=1)
result_df.to_csv('word_dictionary_final.csv', index=False, encoding='utf-8-sig')

In [1]:
import pandas as pd

df = pd.read_csv('word_dictionary_final.csv', dtype=str)

def simplify_word(word):

    if pd.isna(word):
        return word
    
    # 밑줄('_') 기준으로 분리
    parts = word.split('_')
    
    # 첫 번째 요소를 시작으로 연속 중복 제거
    new_parts = [parts[0]]
    for part in parts[1:]:
        if part != new_parts[-1]:
            new_parts.append(part)
    
    # 리스트를 띄어쓰기로 연결하여 반환
    return ' '.join(new_parts)

for col in df.columns:
    df[col] = df[col].apply(simplify_word)

# 결과를 새로운 CSV 파일로 저장 (UTF-8 with BOM: 엑셀 등에서 한글 깨짐 방지)
df.to_csv('word_dictionary_cleaned.csv', index=False, encoding='utf-8-sig')